In [2]:
# Uses code from 'make j from scratch'
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)


    

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def sigmaOdd(n):
    sum=0
    dvs=divisors(n)
    lnth=len(dvs)
    for k in [0..lnth-1]:
        dv=dvs[k]
        if mod(dv,2)==1:
            sum=sum+dv
    return sum

def division(dividend, divisor) : 
    quotient=(dividend._maxima_().divide(divisor).sage())[0] 
    remainder=(dividend._maxima_().divide(divisor).sage())[1] 
    return (quotient,remainder)

def HsubIstrike(n,m):
    jay = J(n+1,m)
    numerator = x^m*derivative(jay,x)^m
    denominator = jay^(m-1)*(jay-1)
    power=1/(m-2)
    answer0=S((numerator/denominator)^power).truncate(n).polynomial(x)
    al=answer0.list()
    dg=answer0.degree()
    sm=sum(al[k]*(2^6*m^3*x)^k for k in [0..dg])*(-1)^m
    return sm

def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel

In [25]:
import pickle
import time
wfile=open('/Users/barrybrent/run11aug21no5test2.txt','w')
data=[]
start=time.time()/60
for m in [3..13]:
    poly=HsubIstrike(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

[3, 99, 0.02397250011563301]
[4, 99, 0.04020193591713905]
[5, 99, 0.07099658250808716]
[6, 99, 0.08639156445860863]
[7, 99, 0.11948895081877708]
[8, 99, 0.14444713294506073]
[9, 99, 0.18071618303656578]
[10, 99, 0.20367396995425224]
[11, 99, 0.23996136710047722]
[12, 99, 0.2711869664490223]
[13, 99, 0.3092659190297127]


In [26]:
import pickle
import time
wfile=open('/Users/barrybrent/run11aug21no6.txt','w')
data=[]
start=time.time()/60
for m in [3..323]:
    poly=HsubIstrike(101,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

[3, 100, 0.02450251579284668]
[4, 100, 0.0409710668027401]
[5, 100, 0.0728084184229374]
[6, 100, 0.08882420137524605]
[7, 100, 0.1229001022875309]
[8, 100, 0.1488635651767254]
[9, 100, 0.18655133247375488]
[10, 100, 0.210385300219059]
[11, 100, 0.24751850217580795]
[12, 100, 0.2798910513520241]
[13, 100, 0.3193850852549076]
[14, 100, 0.3458416201174259]
[15, 100, 0.39092638716101646]
[16, 100, 0.42201799899339676]
[17, 100, 0.4648245647549629]
[18, 100, 0.49413158744573593]
[19, 100, 0.5365835167467594]
[20, 100, 0.5743489824235439]
[21, 100, 0.6209310330450535]
[22, 100, 0.6489348337054253]
[23, 100, 0.6920497827231884]
[24, 100, 0.7311983518302441]
[25, 100, 0.7789809517562389]
[26, 100, 0.8113940693438053]
[27, 100, 0.8592451177537441]
[28, 100, 0.8990842513740063]
[29, 100, 0.943476065993309]
[30, 100, 0.9798315316438675]
[31, 100, 1.024264868348837]
[32, 100, 1.0599342994391918]
[33, 100, 1.1110662184655666]
[34, 100, 1.143815815448761]
[35, 100, 1.1973122172057629]
[36, 100, 1.24

[276, 100, 14.497358717024326]
[277, 100, 14.562211368232965]
[278, 100, 14.612264983355999]
[279, 100, 14.682177133858204]
[280, 100, 14.745582517236471]
[281, 100, 14.812943134456873]
[282, 100, 14.869510766118765]
[283, 100, 14.936571884900331]
[284, 100, 14.996380534023046]
[285, 100, 15.071874096989632]
[286, 100, 15.12397426739335]
[287, 100, 15.194518581032753]
[288, 100, 15.253018885850906]
[289, 100, 15.321936532855034]
[290, 100, 15.375267934054136]
[291, 100, 15.444020483642817]
[292, 100, 15.500529386103153]
[293, 100, 15.566187486052513]
[294, 100, 15.623554065823555]
[295, 100, 15.694975983351469]
[296, 100, 15.753286965191364]
[297, 100, 15.82374331727624]
[298, 100, 15.873519599437714]
[299, 100, 15.94234075397253]
[300, 100, 16.006142236292362]
[301, 100, 16.075279664248228]
[302, 100, 16.12533436715603]
[303, 100, 16.19710698351264]
[304, 100, 16.25530543550849]
[305, 100, 16.32667848467827]
[306, 100, 16.382242549210787]
[307, 100, 16.450627867132425]
[308, 100, 16.5

In [ ]:
import pickle
import time
rfile=open('/Users/barrybrent/run11aug21no6.txt','r')
# i'm going to break rfile into three smaller ones for github.
rfile.close()
s=stripQuotationMarks(rs)

In [4]:
import pickle
rfile=open('/Users/barrybrent/run11aug21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run11jaug21no9.txt','w') 
data=[]
for k in  [0..115]:
    data=data+[s[k]]

t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [5]:
import pickle
rfile=open('/Users/barrybrent/run11aug21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run11jaug21no10.txt','w') 
data=[]
for k in  [116..230]:
    data=data+[s[k]]

t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [6]:
import pickle
rfile=open('/Users/barrybrent/run11aug21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

wfile = open('/Users/barrybrent/run11jaug21no11.txt','w') 
data=[]
for k in  [231..len(s)-1]:
    data=data+[s[k]]

t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [39]:
rfile=open('/Users/barrybrent/run11aug21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


wfile = open('/Users/barrybrent/run11jaug21no8.txt','w') #HsubIstrike polynomials
polydata=[]
start=time.time()/60
for n in [0..100]:
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1]
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    poly=rl.polynomial(x)
    polydata=polydata+[[n,poly]] 
    finish=time.time()/60
    print(n,poly.degree(),finish-start)
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

(0, 0, 0.014840416610240936)
(1, 3, 0.02903289720416069)
(2, 6, 0.043728917837142944)
(3, 9, 0.05880270153284073)
(4, 12, 0.07278798520565033)
(5, 15, 0.08770078420639038)
(6, 18, 0.1017853356897831)
(7, 21, 0.11660593375563622)
(8, 24, 0.13079391792416573)
(9, 27, 0.14573653042316437)
(10, 30, 0.16059578210115433)
(11, 33, 0.17469045147299767)
(12, 36, 0.1898159310221672)
(13, 39, 0.20391588285565376)
(14, 42, 0.21873991563916206)
(15, 45, 0.2340330183506012)
(16, 48, 0.2484411671757698)
(17, 51, 0.26356038451194763)
(18, 54, 0.27764031663537025)
(19, 57, 0.29298199713230133)
(20, 60, 0.30820778384804726)
(21, 63, 0.322998583316803)
(22, 66, 0.3389301672577858)
(23, 69, 0.3542199172079563)
(24, 72, 0.36962058395147324)
(25, 75, 0.38454846665263176)
(26, 78, 0.4004156142473221)
(27, 81, 0.4162410981953144)
(28, 84, 0.4312950484454632)
(29, 87, 0.447103351354599)
(30, 90, 0.4621010683476925)
(31, 93, 0.4780694842338562)
(32, 96, 0.49445991590619087)
(33, 99, 0.510023333132267)
(34, 102,

In [40]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

101

In [3]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..3]:
    n=s[k][0]
    poly=D(s[k][1])
    print "===================================================================================="
    print (k,n)
    print 
    print poly
    print
    print poly.factor()
    

(0, 0)

1

1
(1, 1)

-24*x^3 + 16*x^2

(-24) * (x - 2/3) * x^2
(2, 2)

24*x^6 - 400*x^5 + 928*x^4 - 448*x^3

(24) * (x - 14) * (x - 2) * (x - 2/3) * x^3
(3, 3)

-96*x^9 + 2368/3*x^8 - 58112/9*x^7 + 230912/9*x^6 - 343552/9*x^5 + 142336/9*x^4

(-96) * (x - 2) * (x - 2/3) * x^4 * (x^3 - 50/9*x^2 + 460/9*x - 1112/9)


In [57]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..5]:
    n=s[k][0]
    poly=D(s[k][1])
    print "===================================================================================="
    print "n: ",n
    print 
    print poly.factor()
    print numericalfactor(poly)

n:  0

1
1
n:  1

(-24) * (x - 2/3) * x^2
-24
n:  2

(24) * (x - 14) * (x - 2) * (x - 2/3) * x^3
24
n:  3

(-96) * (x - 2) * (x - 2/3) * x^4 * (x^3 - 50/9*x^2 + 460/9*x - 1112/9)
-96
n:  4

(24) * (x - 2) * (x - 2/3) * x^5 * (x^5 - 298/3*x^4 + 19208/27*x^3 - 90992/27*x^2 + 343600/27*x - 520864/27)
24
n:  5

(-144) * (x - 2) * (x - 2/3) * x^6 * (x^7 - 766/45*x^6 + 633908/2025*x^5 - 5608696/2025*x^4 + 26620592/2025*x^3 - 90708256/2025*x^2 + 45835456/405*x - 268935808/2025)
-144


In [45]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf]
print data 

[1, -24, 24, -96, 24, -144, 96, -192, 24, -312, 144, -288, 96, -336, 192, -576, 24, -432, 312, -480, 144, -768, 288, -576, 96, -744, 336, -960, 192, -720, 576, -768, 24, -1152, 432, -1152, 312, -912, 480, -1344, 144, -1008, 768, -1056, 288, -1872, 576, -1152, 96, -1368, 744, -1728, 336, -1296, 960, -1728, 192, -1920, 720, -1440, 576, -1488, 768, -2496, 24, -2016, 1152, -1632, 432, -2304, 1152, -1728, 312, -1776, 912, -2976, 480, -2304, 1344, -1920, 144, -2904, 1008, -2016, 768, -2592, 1056, -2880, 288, -2160, 1872, -2688, 576, -3072, 1152, -2880, 96, -2352, 1368, -3744, 744]


In [46]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf/24]
print data

[-1, 1, -4, 1, -6, 4, -8, 1, -13, 6, -12, 4, -14, 8, -24, 1, -18, 13, -20, 6, -32, 12, -24, 4, -31, 14, -40, 8, -30, 24, -32, 1, -48, 18, -48, 13, -38, 20, -56, 6, -42, 32, -44, 12, -78, 24, -48, 4, -57, 31, -72, 14, -54, 40, -72, 8, -80, 30, -60, 24, -62, 32, -104, 1, -84, 48, -68, 18, -96, 48, -72, 13, -74, 38, -124, 20, -96, 56, -80, 6, -121, 42, -84, 32, -108, 44, -120, 12, -90, 78, -112, 24, -128, 48, -120, 4, -98, 57, -156, 31]


In [ ]:
# sigmaOdd(n)*(-1)^n  OEIS A000593 auth. NJA Sloane URL = http://oeis.org/A000593

In [49]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf/24-sigmaOdd(n)*(-1)^n]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
f=x^2;f.degree(x)

2

In [50]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    dg=poly.degree() # under D coercion, degree.() takes no argument.
    data=data+[3*n-dg]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [52]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..4]:
    n=s[k][0]
    poly=s[k][1]
    print "===================================================================================="
    print "n: ",n
    print
    print division(poly,x-2)
   

n:  0

(0, 1)
n:  1

(-24*x^2 - 32*x - 64, -128)
n:  2

(24*x^5 - 352*x^4 + 224*x^3, 0)
n:  3

(-96*x^8 + 1792/3*x^7 - 47360/9*x^6 + 136192/9*x^5 - 71168/9*x^4, 0)
n:  4

(24*x^11 - 2400*x^10 + 167968/9*x^9 - 2491136/27*x^8 + 9702272/27*x^7 - 17998336/27*x^6 + 8333824/27*x^5, 0)


In [65]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    
    n=s[k][0]
    print n
    poly=D(s[k][1])
    dv=division(poly,D((x-2)^2))
    if dv[1]==0:data=data+[n]
print data
   

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
[]


In [66]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    
    n=s[k][0]
    print n
    poly=D(s[k][1])
    dv=division(poly,D((x-2/3)^2))
    if dv[1]==0:data=data+[n]
print data
   

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
[]


In [3]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    print n
    poly=D(s[k][1])
    dv=division(poly,D((x^(n+2))))
    if dv[1]==0:
                data=data+[n]
print data

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
[]


In [6]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    divideby=(x-2/3)*(x-2)*x^(n+1)
    g=division(poly,divideby)
    explist=polynomialExponentList(D(g[0]))
    # explist = 1 => g[0] is irreducible.
    data=data+[explist]
print data

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
